# Notebook 05: Intégration et Test d'une Équipe CrewAI (Document Deep Dive Crew)

Ce notebook se concentre sur le test et la démonstration de notre équipe d'agents CrewAI spécialisée dans l'analyse approfondie de documents : la `DocumentAnalysisCrew`. Nous allons instancier cette équipe en utilisant le module `src.agents.crewai_teams.document_analysis_crew`, lui soumettre un exemple de document et un focus de recherche, et examiner le rapport d'analyse qu'elle produit.

L'équipe `DocumentAnalysisCrew` utilise en interne la fonction `get_llm` de `src/llm_services/llm_factory.py` pour instancier le modèle de langage en fonction de la configuration `DEFAULT_LLM_MODEL_PROVIDER` de votre projet.

**Prérequis :**
* **Environnement de Base :** Avoir exécuté le notebook `00_setup_environment.ipynb` pour configurer l'environnement Conda, les dépendances Python (y compris `crewai`, et d'autres comme `langchain-huggingface` si vous utilisez le fournisseur Hugging Face API), et s'assurer que le fichier `.env` à la racine du projet est correctement rempli.
* **Configuration du Fournisseur LLM (dans `.env`) :** La `DocumentAnalysisCrew` utilisera le fournisseur LLM spécifié par `DEFAULT_LLM_MODEL_PROVIDER` (et les variables associées) dans votre fichier `.env`.
    * Si `DEFAULT_LLM_MODEL_PROVIDER` est `"openai"` : `OPENAI_API_KEY` et `DEFAULT_OPENAI_GENERATIVE_MODEL` sont requis.
    * Si `DEFAULT_LLM_MODEL_PROVIDER` est `"huggingface_api"` : `HUGGINGFACE_API_KEY` et `HUGGINGFACE_REPO_ID` sont requis. Assurez-vous que `langchain-huggingface` et `transformers` sont bien installés.
    * Si `DEFAULT_LLM_MODEL_PROVIDER` est `"ollama"` : Assurez-vous que `OLLAMA_BASE_URL` pointe vers votre instance Ollama et que `OLLAMA_GENERATIVE_MODEL_NAME` est un modèle que vous avez téléchargé (`ollama pull ...`) et qui est servi par Ollama.
* **(Optionnel) Base de Données pour Contexte Général :** Bien que ce notebook fournisse directement le contenu du document à analyser à la `DocumentAnalysisCrew`, il est utile de se rappeler que dans un scénario d'utilisation complet, le contenu pourrait provenir d'une base de données peuplée via `01_data_ingestion_and_embedding.ipynb`.

In [1]:
import logging
import sys
from pathlib import Path
import os
import json # Pour un affichage lisible (bien que non utilisé directement dans cette cellule)
from typing import Optional, List 

# --- Activation du mode verbeux de LiteLLM et configuration de l'alias ---
import litellm
litellm.set_verbose = True # Pour obtenir des logs détaillés de LiteLLM

project_root = Path()

from dotenv import load_dotenv
# De même, si CWD est la racine du projet, dotenv_path serait `Path().resolve() / ".env"`
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}. Assurez-vous qu'il est à la racine du projet.")

# Imports des modules du projet APRES la configuration potentielle du PYTHONPATH et de dotenv
from config.settings import settings
from config.logging_config import setup_logging
from src.agents.crewai_teams.document_analysis_crew import run_document_deep_dive_crew

# --- Configuration de l'alias de modèle LiteLLM (APRES que 'settings' soit chargé) ---
# Ceci aide LiteLLM à mapper un nom de modèle simple (ex: "mistral") 
# vers le format préfixé qu'il attend pour Ollama (ex: "ollama/mistral"),
# en particulier si CrewAI ne passe pas le contexte du fournisseur de manière explicite à LiteLLM.
if hasattr(settings, 'DEFAULT_LLM_MODEL_PROVIDER') and settings.DEFAULT_LLM_MODEL_PROVIDER.lower() == "ollama":
    if hasattr(settings, 'OLLAMA_GENERATIVE_MODEL_NAME') and settings.OLLAMA_GENERATIVE_MODEL_NAME:
        configured_ollama_model_name = settings.OLLAMA_GENERATIVE_MODEL_NAME
        # S'assurer que model_alias_map existe avant de l'utiliser
        if not hasattr(litellm, 'model_alias_map') or litellm.model_alias_map is None:
            litellm.model_alias_map = {} # Initialiser si ce n'est pas déjà un dict
            
        litellm.model_alias_map[configured_ollama_model_name] = f"ollama/{configured_ollama_model_name}"
        # On peut aussi logger, mais le logger n'est pas encore configuré.
        print(f"INFO (pre-logging): LiteLLM model alias set: '{configured_ollama_model_name}' -> 'ollama/{configured_ollama_model_name}'")
    else:
        print("WARNING (pre-logging): OLLAMA_GENERATIVE_MODEL_NAME non défini, alias LiteLLM non appliqué.")
# --- Fin de la configuration de l'alias ---

# Configurer le logging pour le notebook
LOG_LEVEL_NOTEBOOK = "INFO" 
setup_logging(level=LOG_LEVEL_NOTEBOOK) 
logger = logging.getLogger("nb_05_crewai_integration") # Le logger est maintenant disponible

# --- Vérification des prérequis pour le LLM utilisé par CrewAI ---
# Cette section logue les configurations pour informer l'utilisateur.
# Les erreurs de configuration critiques sont levées par get_llm() dans llm_factory.py.
active_llm_provider_for_crew = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"L'équipe CrewAI (via DocumentAnalysisCrew -> get_llm) tentera d'utiliser le fournisseur LLM configuré par défaut : '{active_llm_provider_for_crew}'.")

config_ok = False
if active_llm_provider_for_crew == "openai":
    if settings.OPENAI_API_KEY and settings.DEFAULT_OPENAI_GENERATIVE_MODEL:
        logger.info(f"Provider OpenAI sélectionné. Clé API trouvée. Modèle: {settings.DEFAULT_OPENAI_GENERATIVE_MODEL}")
        config_ok = True
    else:
        logger.error("ERREUR : Le fournisseur LLM pour CrewAI est 'openai', mais OPENAI_API_KEY et/ou DEFAULT_OPENAI_GENERATIVE_MODEL ne sont pas (entièrement) configurés.")
        if not settings.OPENAI_API_KEY: print("ERREUR: Clé API OpenAI (OPENAI_API_KEY) manquante dans .env.")
        if not settings.DEFAULT_OPENAI_GENERATIVE_MODEL: print("ERREUR: Modèle OpenAI (DEFAULT_OPENAI_GENERATIVE_MODEL) manquant dans .env/settings.py.")
elif active_llm_provider_for_crew == "huggingface_api":
    if settings.HUGGINGFACE_API_KEY and settings.HUGGINGFACE_REPO_ID:
        logger.info(f"Provider HuggingFace API sélectionné. Clé API trouvée. Repo ID: {settings.HUGGINGFACE_REPO_ID}")
        config_ok = True
    else:
        logger.error("ERREUR : Le fournisseur LLM pour CrewAI est 'huggingface_api', mais HUGGINGFACE_API_KEY et/ou HUGGINGFACE_REPO_ID ne sont pas (entièrement) configurés.")
        if not settings.HUGGINGFACE_API_KEY: print("ERREUR: Clé API HuggingFace (HUGGINGFACE_API_KEY) manquante dans .env.")
        if not settings.HUGGINGFACE_REPO_ID: print("ERREUR: Repo ID HuggingFace (HUGGINGFACE_REPO_ID) manquant dans .env/settings.py.")
elif active_llm_provider_for_crew == "ollama":
    if settings.OLLAMA_BASE_URL and settings.OLLAMA_GENERATIVE_MODEL_NAME:
        logger.info(f"Provider Ollama sélectionné. URL de base: {settings.OLLAMA_BASE_URL}, Modèle: {settings.OLLAMA_GENERATIVE_MODEL_NAME}")
        logger.info(f"ASSUREZ-VOUS que le modèle '{settings.OLLAMA_GENERATIVE_MODEL_NAME}' est disponible sur votre serveur Ollama (via 'ollama pull {settings.OLLAMA_GENERATIVE_MODEL_NAME}').")
        config_ok = True
    else:
        logger.error("ERREUR : Le fournisseur LLM pour CrewAI est 'ollama', mais OLLAMA_BASE_URL et/ou OLLAMA_GENERATIVE_MODEL_NAME ne sont pas (entièrement) configurés.")
        if not settings.OLLAMA_BASE_URL: print("ERREUR: URL de base Ollama (OLLAMA_BASE_URL) manquante dans .env/settings.py.")
        if not settings.OLLAMA_GENERATIVE_MODEL_NAME: print("ERREUR: Nom du modèle génératif Ollama (OLLAMA_GENERATIVE_MODEL_NAME) manquant dans .env/settings.py.")
else:
    logger.error(f"ERREUR : Fournisseur LLM '{active_llm_provider_for_crew}' non reconnu ou non supporté par les vérifications de ce notebook.")
    print(f"ERREUR: Fournisseur LLM '{active_llm_provider_for_crew}' non supporté par ces vérifications. L'exécution de CrewAI pourrait échouer.")

if not config_ok:
    logger.warning("ATTENTION : La configuration pour le fournisseur LLM sélectionné semble incomplète. L'initialisation de l'LLM par CrewAI via get_llm() risque d'échouer.")

# Vérifier si crewai est installé
try:
    import crewai
    logger.info(f"Bibliothèque CrewAI version {crewai.__version__} importée.")
except ImportError:
    logger.error("La bibliothèque 'crewai' n'est PAS installée. Veuillez l'installer (pip install crewai) et relancer.")
    crewai = None 
    print("ERREUR: Bibliothèque 'crewai' non trouvée. Veuillez l'installer.")

Variables d'environnement chargées depuis : .env

2025-06-03 10:16:18 - nb_05_crewai_integration - INFO - L'équipe CrewAI (via DocumentAnalysisCrew -> get_llm) tentera d'utiliser le fournisseur LLM configuré par défaut : 'ollama'.
2025-06-03 10:16:18 - nb_05_crewai_integration - INFO - Provider Ollama sélectionné. URL de base: http://localhost:11434, Modèle: mistral
2025-06-03 10:16:18 - nb_05_crewai_integration - INFO - ASSUREZ-VOUS que le modèle 'mistral' est disponible sur votre serveur Ollama (via 'ollama pull mistral').
2025-06-03 10:16:18 - nb_05_crewai_integration - INFO - Bibliothèque CrewAI version 0.121.1 importée.


### 1. Préparation des Données d'Exemple pour l'Analyse

Nous allons définir un exemple de contenu de document, son ID, et un focus de recherche spécifique pour tester notre `DocumentAnalysisCrew`.

In [2]:
sample_doc_id = "arxiv_sample_2401.001"
sample_research_focus = "Identify the core methodology, key results, and explicitly stated limitations regarding sim-to-real transfer techniques."

sample_doc_content = """
Abstract: This research presents "TransferRL", a novel framework for enhancing sim-to-real transfer in robotic manipulation tasks. 
By employing adaptive instance normalization in the feature space of a vision-based policy and a curriculum of progressively complex simulated environments, 
TransferRL significantly reduces the reality gap. We demonstrate its efficacy on a pick-and-place task, achieving an 82% success rate on a physical KUKA LBR iiwa robot after training purely in simulation.

Introduction: Sim-to-real transfer is a pivotal challenge in applying deep reinforcement learning (RL) to real-world robotics. 
While training RL agents in simulation is often faster, safer, and more scalable than on physical systems, policies learned in simulation frequently
fail to generalize to the real world due to discrepancies in dynamics, visual appearance, and sensor noise. 
This paper addresses these challenges with a focus on visual domain adaptation.

Methodology: TransferRL consists of two main components. First, an image-to-image translation network based on CycleGAN is used to translate
simulated images to a more realistic style, though this is only used for initial policy pre-training. The core of TransferRL is an
encoder-decoder policy architecture where the encoder features are modulated by adaptive instance normalization (AdaIN) layers. 
These AdaIN layers are conditioned on a learned domain embedding, allowing the policy to adapt its visual processing dynamically.
The agent is trained using Soft Actor-Critic (SAC) on a curriculum of tasks in a custom OpenAI Gym environment built with PyBullet. 
The curriculum gradually increases object diversity and complexity of grasp poses. Domain randomization is applied to lighting, textures, and camera positions.

Results: On the primary pick-and-place task, TransferRL (82% success on physical robot) outperformed: 
(a) direct transfer without AdaIN or curriculum (35%), 
(b) AdaIN without curriculum (65%), and 
(c) curriculum without AdaIN (58%). 
The domain embedding was observed to cluster according to different simulation variations, suggesting successful domain identification. 
Qualitative analysis showed improved handling of novel object textures and lighting in the real world.

Limitations: The current implementation of TransferRL requires a small set of real-world images to learn the target domain embedding for AdaIN, making it not entirely "zero-shot" sim-to-real. 
The computational overhead of the AdaIN layers adds a minor latency to the policy execution. 
The system has only been tested on quasi-static pick-and-place tasks.

Conclusion: TransferRL offers an effective method for robust sim-to-real transfer in vision-based robotic manipulation by combining adaptive normalization with a structured training curriculum. Future work will explore fully zero-shot adaptation and application to dynamic tasks.
"""

logger.info(f"Prêt à analyser le document ID: {sample_doc_id}")
logger.info(f"Focus de la recherche: {sample_research_focus}")
# Un log pour la longueur du contenu peut être utile aussi:
# logger.info(f"Longueur du contenu du document échantillon: {len(sample_doc_content)} caractères")

2025-06-03 10:16:18 - nb_05_crewai_integration - INFO - Prêt à analyser le document ID: arxiv_sample_2401.001
2025-06-03 10:16:18 - nb_05_crewai_integration - INFO - Focus de la recherche: Identify the core methodology, key results, and explicitly stated limitations regarding sim-to-real transfer techniques.


### 2. Exécution de l'Équipe CrewAI `DocumentAnalysisCrew`

Nous appelons la fonction `run_document_deep_dive_crew` (définie dans `src/agents/crewai_teams/document_analysis_crew.py`) avec nos données d'exemple. Cette fonction orchestre l'équipe CrewAI pour produire un rapport d'analyse.

In [3]:
final_report_from_crew = None

# La condition principale est maintenant de vérifier si la bibliothèque 'crewai' est disponible.
# Les configurations spécifiques au LLM (OpenAI, HuggingFace API, Ollama) sont gérées par:
# 1. Les messages d'erreur/logs dans la première cellule de code de ce notebook (ID '1a6fa31a').
# 2. La fonction get_llm() (dans src/llm_services/llm_factory.py) qui lève une ValueError si la configuration est incorrecte.
# 3. Le bloc try...except ci-dessous qui attrape ces erreurs lors de l'exécution.

if crewai: # 'crewai' est l'objet module importé dans la première cellule (ID '1a6fa31a').
    logger.info("Tentative de lancement de DocumentAnalysisCrew...")
    # Les variables sample_doc_id, sample_doc_content, sample_research_focus 
    # sont définies dans la cellule précédente (ID '9ed249b5').
    print(f"\n🔄 Lancement de l'équipe CrewAI pour l'analyse du document ID: {sample_doc_id}...")
    print(f"   Focus de la recherche: {sample_research_focus}")
    print(f"   Utilisation du fournisseur LLM: '{settings.DEFAULT_LLM_MODEL_PROVIDER}' (Modèle: '{settings.OLLAMA_GENERATIVE_MODEL_NAME if settings.DEFAULT_LLM_MODEL_PROVIDER == 'ollama' else (settings.HUGGINGFACE_REPO_ID if settings.DEFAULT_LLM_MODEL_PROVIDER == 'huggingface_api' else settings.DEFAULT_OPENAI_GENERATIVE_MODEL)}')")
    print("   Cela peut prendre du temps...\n")
    
    try:
        # Appel à la fonction qui exécute l'équipe CrewAI.
        # Cette fonction utilise DocumentAnalysisCrew, qui utilise get_llm()
        # pour instancier le LLM selon la configuration du projet.
        final_report_from_crew = run_document_deep_dive_crew(
            document_id=sample_doc_id,
            document_content=sample_doc_content,
            research_focus=sample_research_focus
        )
        
        print("\n\n--- 📝 Rapport d'Analyse Final Compilé par l'Équipe CrewAI ---")
        if final_report_from_crew:
            # Vérifier si le rapport est un message d'erreur géré par DocumentAnalysisCrew.run()
            if isinstance(final_report_from_crew, str) and final_report_from_crew.lower().startswith("error:"):
                 print(f"L'équipe CrewAI a retourné une erreur : {final_report_from_crew}")
            else:
                # Tenter d'afficher en Markdown si possible, sinon en texte brut.
                try:
                    from IPython.display import display, Markdown
                    # S'assurer que le contenu est une chaîne avant de le passer à Markdown
                    display(Markdown(str(final_report_from_crew))) 
                except ImportError:
                    print(str(final_report_from_crew)) 
        else:
            print("L'équipe CrewAI n'a pas retourné de rapport explicite (vérifiez les logs pour des erreurs).")

    except ValueError as ve: # Attrape les erreurs de configuration de get_llm (ex: clé API manquante pour le provider choisi)
        logger.error(f"Erreur de configuration LLM lors de l'exécution de DocumentAnalysisCrew: {ve}", exc_info=True)
        print(f"ERREUR DE CONFIGURATION LLM (via get_llm) : {ve}")
        print(f"Veuillez vérifier la configuration pour le provider '{settings.DEFAULT_LLM_MODEL_PROVIDER}' dans votre .env.")
    except Exception as e: # Capture les autres erreurs pendant l'exécution de la Crew.
        logger.error(f"Erreur lors de l'exécution de DocumentAnalysisCrew dans le notebook: {e}", exc_info=True)
        print(f"ERREUR lors de l'exécution de l'équipe CrewAI : {e}")
else:
    # Ce bloc est atteint si l'import de 'crewai' a échoué dans la première cellule de code.
    print("Bibliothèque CrewAI non installée ou import échoué. Test de l'équipe CrewAI sauté.")
    logger.warning("Skipping CrewAI team test because 'crewai' library is not available.")

2025-06-03 10:16:18 - nb_05_crewai_integration - INFO - Tentative de lancement de DocumentAnalysisCrew...

🔄 Lancement de l'équipe CrewAI pour l'analyse du document ID: arxiv_sample_2401.001...
   Focus de la recherche: Identify the core methodology, key results, and explicitly stated limitations regarding sim-to-real transfer techniques.
   Utilisation du fournisseur LLM: 'ollama' (Modèle: 'mistral')
   Cela peut prendre du temps...

2025-06-03 10:16:18 - src.llm_services.llm_factory - INFO - Initializing LLM from llm_factory for provider: 'ollama' with temperature: 0.2
2025-06-03 10:16:18 - src.llm_services.llm_factory - INFO - Using Ollama model (via langchain_ollama): mistral from http://localhost:11434
2025-06-03 10:16:18 - src.agents.crewai_teams.document_analysis_crew - INFO - Starting Document Analysis Crew for doc ID: arxiv_sample_2401.001, focus: 'Identify the core methodology, key results, and explicitly stated limitations regarding sim-to-real transfer techniques.'


10:16:18 - LiteLLM:WARNING: utils.py:511 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


# Agent: Expert Information Extractor for Scientific Papers
## Task: Analyze the following document (ID: arxiv_sample_2401.001) with a research focus on 'Identify the core methodology, key results, and explicitly stated limitations regarding sim-to-real transfer techniques.'. Extract key information: primary methodology, datasets used (if any), main quantitative results, and author-stated limitations. Present this as a structured list or key-value pairs. Document Content:

---

Abstract: This research presents "TransferRL", a novel framework for enhancing sim-to-real transfer in robotic manipulation tasks. 
By employing adaptive instance normalization in the feature space of a vision-based policy and a curriculum of progressively complex simulated environments, 
TransferRL significantly reduces the reality gap. We demonstrate its efficacy on a pick-and-place task, achieving an 82% success rate on a physical KUKA LBR iiwa robot after training purely in simulation.

Introduction: Sim-to-

10:16:18 - LiteLLM:INFO: utils.py:2827 - 
LiteLLM completion() model= mistral; provider = ollama




Final returned optional params: {'temperature': 0.2, 'stream': False, 'stop': ['\nObservation:']}
2025-06-03 10:16:26 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


...

2025-06-03 10:17:26 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
2025-06-03 10:17:26 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


# Agent: Lead Research Report Compiler
## Final Answer: 
The comprehensive analytical report for document arxiv_sample_2401.001 covering 1. Extracted Key Information (Methodology, Results, Limitations). 2. Section Summaries. 3. Critical Analysis (Strengths, Weaknesses, Contributions) is as follows:

   The paper presents "TransferRL", a novel framework for enhancing sim-to-real transfer in robotic manipulation tasks. The methodology involves two main components: an image-to-image translation network based on CycleGAN for initial policy pre-training and an encoder-decoder policy architecture with adaptive instance normalization (AdaIN) layers conditioned on a learned domain embedding. The agent is trained using Soft Actor-Critic (SAC) in a custom OpenAI Gym environment built w

10:17:26 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: ollama/mistral


2025-06-03 10:17:26 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
2025-06-03 10:17:26 - src.agents.crewai_teams.document_analysis_crew - INFO - Document Analysis Crew finished for doc ID: arxiv_sample_2401.001. Result length: 2303


--- 📝 Rapport d'Analyse Final Compilé par l'Équipe CrewAI ---


The comprehensive analytical report for document arxiv_sample_2401.001 covering 1. Extracted Key Information (Methodology, Results, Limitations). 2. Section Summaries. 3. Critical Analysis (Strengths, Weaknesses, Contributions) is as follows:

   The paper presents "TransferRL", a novel framework for enhancing sim-to-real transfer in robotic manipulation tasks. The methodology involves two main components: an image-to-image translation network based on CycleGAN for initial policy pre-training and an encoder-decoder policy architecture with adaptive instance normalization (AdaIN) layers conditioned on a learned domain embedding. The agent is trained using Soft Actor-Critic (SAC) in a custom OpenAI Gym environment built with PyBullet, following a curriculum of tasks that gradually increases object diversity and complexity of grasp poses. Domain randomization is applied to lighting, textures, and camera positions.

   The key results demonstrate the effectiveness of TransferRL, as it achieved an 82% success rate on a physical KUKA LBR iiwa robot after training purely in simulation, outperforming various control strategies without AdaIN or curriculum. The domain embedding was observed to cluster according to different simulation variations, suggesting successful domain identification. Qualitative analysis showed improved handling of novel object textures and lighting in the real world.

   However, there are some explicitly stated limitations. First, the current implementation of TransferRL requires a small set of real-world images to learn the target domain embedding for AdaIN, making it not entirely "zero-shot" sim-to-real. Second, the computational overhead of the AdaIN layers adds a minor latency to the policy execution. Lastly, the system has only been tested on quasi-static pick-and-place tasks.

   In terms of novel contributions, TransferRL combines adaptive normalization with a structured training curriculum to offer an effective method for robust sim-to-real transfer in vision-based robotic manipulation. Future work will explore fully zero-shot adaptation and application to dynamic tasks. The paper's focus on visual domain adaptation and the use of AdaIN layers to dynamically adapt the policy to different environments are notable contributions to the field.

2025-06-03 10:17:26 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


### 3. Analyse du Rapport et Discussion

* Examinez attentivement le rapport généré par l'équipe CrewAI.
* **Structure :** Le `ReportCompilerAgent` a-t-il bien structuré le rapport en fonction des contributions des autres agents (Extracteur, Résumeur, Analyste Critique) ?
* **Contenu :**
  * Les informations clés extraites par l'`InfoExtractorAgent` (méthodologie, résultats, limitations) sont-elles précises et pertinentes par rapport au `research_focus` ?
  * Les résumés de sections (si l'`SectionSummarizerAgent` a pu les produire) sont-ils concis et informatifs ?
  * L'analyse critique du `CriticalAnalystAgent` (forces, faiblesses, contributions) est-elle judicieuse ?
* **Focus de Recherche :** Le rapport final est-il bien axé sur le `research_focus` que nous avons spécifié ?
* **Verbosité de CrewAI :**
    * Le paramètre `verbose` pour l'objet `Crew` principal dans `src/agents/crewai_teams/document_analysis_crew.py` est actuellement réglé sur `False` pour éviter les problèmes d'affichage des codes de couleur ANSI dans certains terminaux. Si vous souhaitez observer le déroulement détaillé des tâches de la `Crew`, vous pouvez le régler temporairement sur `True`.
    * Les agents individuels (`InfoExtractorAgent`, `SectionSummarizerAgent`, etc.) sont toujours configurés avec `verbose=True`. Si la verbosité de la `Crew` est également sur `True`, vous verrez le détail de leurs "pensées" et actions, ce qui est utile pour comprendre leur processus de collaboration et déboguer.
* **Fournisseur LLM :** Le rapport a été généré en utilisant le fournisseur LLM configuré par `DEFAULT_LLM_MODEL_PROVIDER` (par exemple, Ollama avec "mistral"). La qualité et le style du rapport peuvent varier en fonction du modèle utilisé.

Cette exécution isolée nous donne une idée de la capacité de l'équipe CrewAI à traiter une tâche d'analyse de document complexe et spécialisée avec le LLM configuré.

### 4. Intégration Actuelle dans le Workflow LangGraph

Cette équipe CrewAI est intégrée comme un outil spécialisé au sein de notre système LangGraph principal. Voici comment cela fonctionne :

1.  **La CrewAI comme Outil LangChain (`document_deep_dive_analysis_tool`) :**
    La fonction `run_document_deep_dive_crew` (qui exécute l'équipe d'analyse de document) est encapsulée en tant qu'outil LangChain personnalisé à l'aide du décorateur `@tool`. Cet outil, nommé `document_deep_dive_analysis_tool`, est défini dans `src/agents/tool_definitions.py`.

    Voici un aperçu de sa structure (le code complet est dans `tool_definitions.py`) :
    ```python
    # Extrait de src/agents/tool_definitions.py :
    from langchain_core.tools import tool
    from src.agents.crewai_teams.document_analysis_crew import run_document_deep_dive_crew

    @tool
    def document_deep_dive_analysis_tool(
        document_id: str, 
        document_content: str, 
        research_focus: str
    ) -> str:
        """
        Performs an in-depth analysis of a single scientific document's text content 
        using a specialized team of AI agents (CrewAI).
        Use this tool when a detailed, structured report focusing on specific aspects 
        of a document is required.
        # Args, Returns...
        """
        return run_document_deep_dive_crew(document_id, document_content, research_focus)
    ```

2.  **Utilisation par l'Agent LangGraph `DocumentAnalysisAgent` :**
    L'agent `DocumentAnalysisAgent`, défini dans `src/agents/agent_architectures.py`, est maintenant équipé de cet outil `document_deep_dive_analysis_tool` (en plus de son outil `knowledge_base_retrieval_tool`).
    Son prompt système (la constante `DOCUMENT_ANALYSIS_SYSTEM_PROMPT_V2` dans `agent_architectures.py`) a été mis à jour pour le guider sur les situations où il est approprié de déléguer une analyse approfondie à cette équipe CrewAI spécialisée. Typiquement, cela se produit lorsqu'un rapport très structuré et détaillé sur un document unique est demandé, avec un focus de recherche précis.

3.  **Flux d'Exécution dans LangGraph :**
    * Le `ResearchPlannerAgent` peut, dans son plan de recherche, identifier un document clé qui nécessite une analyse poussée et spécifier un focus pour cette analyse.
    * Le routeur dans LangGraph dirige ensuite la tâche vers le `DocumentAnalysisAgent`.
    * Le `DocumentAnalysisAgent`, en se basant sur le plan ou la requête, peut déterminer qu'une analyse en profondeur est nécessaire. Avant d'appeler `document_deep_dive_analysis_tool`, il pourrait avoir besoin de récupérer le contenu complet du document spécifié (par exemple, en utilisant `knowledge_base_retrieval_tool` pour agréger tous les chunks d'un `arxiv_id` particulier).
    * Une fois le contenu et le focus prêts, il appelle `document_deep_dive_analysis_tool`.
    * Le `ToolNode` de LangGraph exécute cet outil, ce qui lance l'ensemble de l'équipe CrewAI.
    * Le rapport final produit par l'équipe CrewAI est retourné au `DocumentAnalysisAgent` sous forme d'un `ToolMessage` dans l'état du graphe LangGraph.
    * Le `DocumentAnalysisAgent` peut alors utiliser ce rapport détaillé pour formuler sa propre réponse, la résumer, ou la transmettre (potentiellement avec d'autres informations) au `SynthesisAgent` pour l'intégration dans le rapport final.

Cette approche hybride permet de combiner la flexibilité et la gestion d'état robuste de LangGraph pour l'orchestration générale du "MAKERS" avec la puissance de la collaboration structurée et orientée rôles de CrewAI pour des tâches d'analyse de documents très spécifiques et complexes.

## Conclusion

Ce notebook a démontré comment instancier et exécuter notre `DocumentAnalysisCrew` pour effectuer une analyse approfondie d'un document. Nous avons également discuté des pistes pour intégrer cette équipe CrewAI comme un composant spécialisé au sein de notre architecture LangGraph principale.

L'utilisation de CrewAI pour des sous-tâches complexes peut enrichir les capacités de notre "MAKERS".